In [10]:
import pyomo.environ as pyo
from pyomo.environ import (
    ConcreteModel, Set, Param, Var, Constraint, Objective, NonNegativeReals,
    Binary, minimize
)
from pyomo.opt import SolverFactory

##############################################################################
# 1) DATA DEFINITIONS
##############################################################################

# Stations and Time Periods
stations = ['S1', 'S2', 'S3', 'S4', 'S5']
time_periods = [1, 2, 3, 4]

# Station Capacities
C = {
    'S1': 8,
    'S2': 12,
    'S3': 10,
    'S4': 5,
    'S5': 8,
}

# Forecasted Demand (fhat)
fhat_data = {
    ('S1',1): 3, ('S1',2): 4, ('S1',3): 5, ('S1',4): 4,
    ('S2',1): 6, ('S2',2): 8, ('S2',3): 9, ('S2',4): 7,
    ('S3',1): 2, ('S3',2): 3, ('S3',3): 5, ('S3',4): 5,
    ('S4',1): 1, ('S4',2): 2, ('S4',3): 2, ('S4',4): 2,
    ('S5',1): 5, ('S5',2): 5, ('S5',3): 6, ('S5',4): 6,
}

# Robust demand deviations (delta)
delta_data = {
    ('S1',1): 0, ('S1',2): 1, ('S1',3): 0, ('S1',4): 0,
    ('S2',1): 1, ('S2',2): 2, ('S2',3): 2, ('S2',4): 1,
    ('S3',1): 0, ('S3',2): 1, ('S3',3): 0, ('S3',4): 0,
    ('S4',1): 0, ('S4',2): 0, ('S4',3): 0, ('S4',4): 0,
    ('S5',1): 1, ('S5',2): 1, ('S5',3): 2, ('S5',4): 2,
}

# Transport costs: c_trans(s1, s2)
c_trans_dict = {
    ('S1','S2'): 4.0,  ('S1','S3'): 6.0,  ('S1','S4'): 9.0,  ('S1','S5'): 7.0,
    ('S2','S1'): 4.0,  ('S2','S3'): 2.0,  ('S2','S4'): 5.0,  ('S2','S5'): 3.0,
    ('S3','S1'): 6.0,  ('S3','S2'): 2.0,  ('S3','S4'): 7.0,  ('S3','S5'): 5.0,
    ('S4','S1'): 9.0,  ('S4','S2'): 5.0,  ('S4','S3'): 7.0,  ('S4','S5'): 8.0,
    ('S5','S1'): 7.0,  ('S5','S2'): 3.0,  ('S5','S3'): 5.0,  ('S5','S4'): 8.0,
}

# Cost parameters
c_lost  = 12.0   # penalty per unit lost demand
c_empty = 10.0   # penalty if station ends empty
lambda_1 = 1.0   # weight for transport cost
lambda_2 = 1.0   # weight for lost demand
lambda_3 = 0.8   # weight for empty-station penalty

# Budget of uncertainty (Gamma)
Gamma_val = 2

##############################################################################
# 2) BUILDING THE MODEL
##############################################################################

model = ConcreteModel("BikeRebalancingModel_V2_Full")

# --- Sets ---
model.S = Set(initialize=stations)
model.T = Set(initialize=time_periods)

# We'll build a set of station pairs (s1,s2) from c_trans_dict
pairs = list(c_trans_dict.keys())
model.SS = Set(dimen=2, initialize=pairs)

# --- Parameters ---
model.C = Param(model.S, initialize=C)

def fhat_init(m, s, t):
    return fhat_data[(s, t)]
model.fhat = Param(model.S, model.T, initialize=fhat_init)

def delta_init(m, s, t):
    return delta_data[(s, t)]
model.delta = Param(model.S, model.T, initialize=delta_init, default=0.0)

model.c_trans = Param(model.SS, initialize=c_trans_dict, default=9999.0)
model.c_lost   = Param(initialize=c_lost)
model.c_empty  = Param(initialize=c_empty)
model.lambda1  = Param(initialize=lambda_1)
model.lambda2  = Param(initialize=lambda_2)
model.lambda3  = Param(initialize=lambda_3)
model.Gamma    = Param(initialize=Gamma_val)

# --- Decision Variables ---

# 1) Mechanical bikes
model.dMech = Var(model.S, model.T, domain=NonNegativeReals, doc="Mech bike inventory")
model.rMech = Var(model.SS, model.T, domain=NonNegativeReals, doc="Mech bikes rebalancing flow")

# 2) E-bikes: Low, Medium, Full, In-transit
model.dLow  = Var(model.S, model.T, domain=NonNegativeReals)
model.dMed  = Var(model.S, model.T, domain=NonNegativeReals)
model.dFull = Var(model.S, model.T, domain=NonNegativeReals)
model.dIn   = Var(model.S, model.T, domain=NonNegativeReals)
model.rE    = Var(model.SS, model.T, domain=NonNegativeReals)

# 3) Charging transitions
model.yLM = Var(model.S, model.T, domain=NonNegativeReals, doc="Low->Med transition")
model.yMF = Var(model.S, model.T, domain=NonNegativeReals, doc="Med->Full transition")

# 4) Demand / Lost Demand
model.x   = Var(model.S, model.T, domain=NonNegativeReals, doc="Rides fulfilled")
model.ell = Var(model.S, model.T, domain=NonNegativeReals, doc="Lost demand")

# 5) Robust usage factor
model.z   = Var(model.S, model.T, bounds=(0,1))

# 6) Empty station indicator
model.alpha = Var(model.S, model.T, domain=Binary)

##############################################################################
# 3) OBJECTIVE FUNCTION
##############################################################################

def obj_rule(m):
    # Transport cost
    transport_cost = sum(
        m.c_trans[s1, s2] * (m.rMech[s1, s2, t] + m.rE[s1, s2, t])
        for (s1, s2) in m.SS for t in m.T
    )
    # Lost demand cost
    lost_demand_cost = sum(m.c_lost * m.ell[s, t] for s in m.S for t in m.T)
    # Empty-station penalty
    empty_penalty = sum(m.c_empty * m.alpha[s, t] for s in m.S for t in m.T)

    return (m.lambda1 * transport_cost
            + m.lambda2 * lost_demand_cost
            + m.lambda3 * empty_penalty)

model.OBJ = Objective(rule=obj_rule, sense=minimize)

##############################################################################
# 4) CONSTRAINTS
##############################################################################

# (A) Demand Satisfaction
def demand_balance_rule(m, s, t):
    return m.x[s,t] + m.ell[s,t] == m.fhat[s,t]
model.DemandBalance = Constraint(model.S, model.T, rule=demand_balance_rule)

# (B) Robust Constraints
def robust_lower_rule(m, s, t):
    # x[s,t] >= fhat[s,t] + delta[s,t] * z[s,t]
    return m.x[s,t] >= m.fhat[s,t] + m.delta[s,t]*m.z[s,t]
model.RobustLower = Constraint(model.S, model.T, rule=robust_lower_rule)

def robust_budget_rule(m):
    # sum of z[s,t] <= Gamma
    return sum(m.z[s,t] for s in m.S for t in m.T) <= m.Gamma
model.RobustBudget = Constraint(rule=robust_budget_rule)

# (C) Station Capacity
def station_capacity_rule(m, s, t):
    return (m.dMech[s,t]
            + m.dLow[s,t]
            + m.dMed[s,t]
            + m.dFull[s,t]) <= m.C[s]
model.StationCap = Constraint(model.S, model.T, rule=station_capacity_rule)

# (D) Usable Bikes to Fulfill Demand
def fulfill_rule(m, s, t):
    # x[s,t] <= dMech[s,t] + dMed[s,t] + dFull[s,t]
    return m.x[s,t] <= m.dMech[s,t] + m.dMed[s,t] + m.dFull[s,t]
model.Fulfill = Constraint(model.S, model.T, rule=fulfill_rule)

# (E) Empty-Station Penalty
def empty_station_rule(m, s, t):
    # If alpha[s,t] = 1 => station has ~0 usable bikes
    # Use a large M (9999) to "turn off" this constraint if alpha=0
    return (m.dMech[s,t] + m.dMed[s,t] + m.dFull[s,t]) <= 9999*(1 - m.alpha[s,t])
model.EmptyStation = Constraint(model.S, model.T, rule=empty_station_rule)

# (F) Mechanical Inventory (toy example)
def mech_inv_rule(m, s, t):
    if t == m.T.first():
        # Skip or initialize at t=1
        return pyo.Constraint.Skip
    else:
        inbound = sum(m.rMech[s1, s, t-1] for (s1, s2) in m.SS if s2 == s)
        outbound = sum(m.rMech[s, s2, t-1] for (s1, s2) in m.SS if s1 == s)
        return m.dMech[s,t] == (
            m.dMech[s,t-1]
            - 0.5*m.x[s,t-1]
            + inbound
            - outbound
        )

model.MechInv = Constraint(model.S, model.T, rule=mech_inv_rule)

# (G) E-Bike In-transit Arrivals
def in_trans_rule(m, s, t):
    if t == m.T.first():
        return m.dIn[s, t] == 0  # no arrivals at t=1
    else:
        return m.dIn[s, t] == sum(m.rE[s1, s, t-1] for (s1, s2) in m.SS if s2 == s)
model.EInTransit = Constraint(model.S, model.T, rule=in_trans_rule)

# (H) E-Bike Outflow Limit
def e_outflow_rule(m, s, t):
    # sum of rE[s, s2, t] over all s2 where (s, s2) ∈ m.SS
    lhs = sum(m.rE[s, s2, t] for (s1, s2) in m.SS if s1 == s)
    rhs = m.dLow[s,t] + m.dMed[s,t] + m.dFull[s,t]
    return lhs <= rhs

model.EOutflow = Constraint(model.S, model.T, rule=e_outflow_rule)

# (I) (Optional) Charging Transitions
# If you want a detailed Low->Med or Med->Full flow in next period,
# you can define additional constraints here.

##############################################################################
# 5) SOLVE THE MODEL
##############################################################################

solver = SolverFactory('cbc')  # or 'glpk','gurobi','cplex', etc.
results = solver.solve(model, tee=True)
print(results)

##############################################################################
# 6) PRINT RESULTS
##############################################################################

# Demand Fulfillment & Lost Demand
for s in model.S:
    for t in model.T:
        print(f"Station {s}, time {t}:")
        print(f"  Demand Fulfilled x={model.x[s,t].value:.2f}")
        print(f"  Lost Demand ell={model.ell[s,t].value:.2f}")
        print(f"  alpha={model.alpha[s,t].value}")
        print(f"  Mech={model.dMech[s,t].value:.2f}, Low={model.dLow[s,t].value:.2f}, "
              f"Med={model.dMed[s,t].value:.2f}, Full={model.dFull[s,t].value:.2f}, In={model.dIn[s,t].value:.2f}")
    print("")

# Rebalancing flows
print("Rebalancing Decisions (Mechanical / E-bikes):")
for (s1, s2) in model.SS:
    for t in model.T:
        valM = model.rMech[s1, s2, t].value
        valE = model.rE[s1, s2, t].value
        if (valM > 1e-4) or (valE > 1e-4):
            print(f"  t={t}, from {s1} to {s2}:  Mech={valM:.2f}, E={valE:.2f}")

# If robust approach was used, check the z variables
print("\nRobust Variables z[s,t]:")
for s in model.S:
    for t in model.T:
        zval = model.z[s,t].value
        if zval > 1e-4:
            print(f"  z[{s},{t}]={zval:.2f}")

print("\nDONE.")


cbc


ApplicationError: No executable found for solver 'cbc'